Давайте создадим простую нейронную сеть, которая будет классифицировать набор данных о цветках ириса. Ниже приведен блок кода для создания простой нейронной сети:

In [1]:
import pandas as pd

dataset = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'])

dataset['species'] = pd.Categorical(dataset['species']).codes

dataset = dataset.sample(frac=1, random_state=1234)

# split the data set into train and test subsets
train_input = dataset.values[:120, :4]
train_target = dataset.values[:120, 4]

test_input = dataset.values[120:, :4]
test_target = dataset.values[120:, 4]

Предыдущий код представляет собой стандартный код, который загружает CSV-файл набора данных Iris, а затем загружает его во фрейм данных pandas. Затем мы перетасовываем строки фрейма данных и разбиваем код на массивы numpy, train_input/train_target (свойства цветка/класс цветка) для обучающих данных и test_input/test_target для тестовых данных. 
Мы будем использовать 120 образцов для обучения и 30 для тестирования. Если вы не знакомы с pandas, думайте об этом как о продвинутой версии NumPy.

Давайте определим нашу первую нейронную сеть. Мы будем использовать сеть прямой связи с одним скрытым слоем с пятью блоками, функцию активации ReLU (это просто другой тип активации, определяемый просто как *f(x) = max(0, x)*) и выходной слой с тремя блоками. Выходной слой состоит из трех блоков, причем каждый блок соответствует одному из трех классов цветка ириса. Ниже приведено определение высоты тона сети:

In [2]:
import torch

torch.manual_seed(1234)

hidden_units = 5

net = torch.nn.Sequential(
    torch.nn.Linear(4, hidden_units), # we'll use a network with 4 hidden units
    torch.nn.ReLU(), # ReLU activation
    torch.nn.Linear(hidden_units, 3) # 3 output units for each of the 3 possible classes
)

Мы будем использовать однократное кодирование для целевых данных. Это означает, что каждый класс цветка будет представлен в виде массива (Iris Setosa = [1, 0, 0], Iris Versicolour = [0, 1, 0] и Iris Virginica = [0, 0, 1]), и один элемент массива будет целевым для одной единицы выходного слоя. Когда сеть классифицирует новый образец, мы определяем класс, беря единицу с наибольшим значением активации. 
`torch.manual_seed(1234)` позволяет нам каждый раз использовать одни и те же случайные данные для воспроизводимости результатов.

Далее мы выберем функцию потерь:

In [3]:
criterion = torch.nn.CrossEntropyLoss()

С помощью переменной `criterion` мы определяем функцию потерь, которую будем использовать, в данном случае это кросс-энтропийные потери. Функция потерь будет измерять, насколько выходные данные сети отличаются от целевых данных.

Затем мы определяем оптимизатор стохастического градиентного спуска (SGD) (вариация алгоритма градиентного спуска) со скоростью обучения 0,1 и импульсом 0,9:

In [4]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9)

Теперь давайте обучим сеть:

In [5]:
epochs = 50

for epoch in range(epochs):
    inputs = torch.autograd.Variable(torch.Tensor(train_input).float())
    targets = torch.autograd.Variable(torch.Tensor(train_target).long())

    optimizer.zero_grad()
    out = net(inputs)
    loss = criterion(out, targets)
    loss.backward()
    optimizer.step()

    if epoch == 0 or (epoch + 1) % 10 == 0:
        print('Epoch %d Loss: %.4f' % (epoch + 1, loss.item()))

Epoch 1 Loss: 1.2181
Epoch 10 Loss: 0.6745
Epoch 20 Loss: 0.2447
Epoch 30 Loss: 0.1397
Epoch 40 Loss: 0.1001
Epoch 50 Loss: 0.0855


Мы проведем обучение в течение 50 эпох, что означает, что мы выполним итерацию 50 раз по набору обучающих данных: 


1. Создайте переменные torch, которые являются `input` и `target`, из массивов numpy train_input и train_target. 
2. Обнулите градиенты оптимизатора, чтобы предотвратить накопление результатов предыдущих итераций. Мы передаем обучающие данные в сеть нейронной сети (входные данные) и вычисляем критерий функции потерь (выход, цели) между выходными данными сети и целевыми данными.
3. Передайте значение потерь обратно по сети. Мы делаем это для того, чтобы можно было рассчитать, как вес каждой сети влияет на функцию потерь. 
4. Оптимизатор обновляет веса сети таким образом, чтобы уменьшить будущие значения функции потерь.

Давайте посмотрим, какова конечная точность нашей модели:

In [6]:
import numpy as np

inputs = torch.autograd.Variable(torch.Tensor(test_input).float())
targets = torch.autograd.Variable(torch.Tensor(test_target).long())

optimizer.zero_grad()
out = net(inputs)
_, predicted = torch.max(out.data, 1)

error_count = test_target.size - np.count_nonzero((targets == predicted).numpy())
print('Errors: %d; Accuracy: %d%%' % (error_count, 100 * torch.sum(targets == predicted) / test_target.size))

Errors: 0; Accuracy: 100%
